This notebook is for converting a TF Saved Graph (saved using a `Saver`) to TF Frozen Graph

## Setup

In [1]:
%%bash

echo 'Train:'
export TRAIN_DIR=../../../tmp/trt_end_to_end/train
find ${TRAIN_DIR} -type f
echo
echo 'Convert (Saver):'
export CONVERT_TF_SAVER_DIR=../../../tmp/trt_end_to_end/convert/tf/saver
find ${CONVERT_TF_SAVER_DIR} -type f | sort
echo
echo 'Convert (Frozen Graph):'
export CONVERT_TF_FROZEN_GRAPH_DIR=../../../tmp/trt_end_to_end/convert/tf/frozen
find ${CONVERT_TF_FROZEN_GRAPH_DIR} -type f | sort

Train:
../../../tmp/trt_end_to_end/train/conv/001/conv_epoch001_2019-09-03T19:30.h5
../../../tmp/trt_end_to_end/train/conv/001/conv_epoch001_2019-09-03T19:30.md
../../../tmp/trt_end_to_end/train/conv/001/params.json
../../../tmp/trt_end_to_end/train/basic/001/params.json
../../../tmp/trt_end_to_end/train/basic/001/basic_epoch001_2019-09-03T19:15.md
../../../tmp/trt_end_to_end/train/basic/001/basic_epoch001_2019-09-03T19:15.h5
../../../tmp/trt_end_to_end/train/batchn/001/batchn_epoch001_2019-09-03T19:28.md
../../../tmp/trt_end_to_end/train/batchn/001/params.json
../../../tmp/trt_end_to_end/train/batchn/001/batchn_epoch001_2019-09-03T19:28.h5
../../../tmp/trt_end_to_end/train/resnet50/001/params.json
../../../tmp/trt_end_to_end/train/resnet50/001/resnet50_epoch001_2019-09-03T19:31.h5
../../../tmp/trt_end_to_end/train/resnet50/001/resnet50_epoch001_2019-09-03T19:31.md

Convert (Saver):
../../../tmp/trt_end_to_end/convert/tf/saver/basic/001/checkpoint
../../../tmp/trt_end_to_end/convert/tf

In [2]:
import sys
sys.path.append('../../..')

## Parameters

In [3]:
_NAME = 'resnet50'
_EPOCH = 1

In [4]:
from src.utils.trt_end_to_end_constants import *
_NAME, _EPOCH, _TIME = get_params(_NAME, _EPOCH)

## Import

In [5]:
# import the needed libraries
import tensorflow as tf
import tensorflow.contrib.tensorrt as trt
from tensorflow.python.platform import gfile
print(tf.__name__, tf.__version__, sep='-')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

tensorflow-1.14.0


In [6]:
from src.utils.trt_end_to_end_constants import *

md_filename = MD_FILE_FORMAT % (_NAME, _EPOCH, _TIME)
_train_dir = get_train_dir(_NAME, _EPOCH)
md_filepath = os.path.join(_train_dir, md_filename)
print(md_filepath)

../../../tmp/trt_end_to_end/train/resnet50/001/resnet50_epoch001_2019-09-03T19:31.md


In [7]:
from src.meta.metadata import Metadata
ret, metadata = Metadata.from_md(md_filepath)
assert ret == 0

from pprint import pprint
pprint(vars(metadata))

{'epoch': 1,
 'input_names': ['resnet50_input:0'],
 'name': 'resnet50',
 'output_names': ['fc100/Softmax:0']}


## Freeze Graph

Input: Saved Graph

In [8]:
saved_graph_dir = get_saver_dir(_NAME, _EPOCH)

saver_filename = SAVED_GRAPH_FILE_FORMAT % (_NAME, _EPOCH, _TIME)
saver_filepath = os.path.join(saved_graph_dir, saver_filename)
print(saver_filepath)

meta_filename = SAVED_GRAPH_META_FILE_FORMAT % (_NAME, _EPOCH, _TIME)
meta_filepath = os.path.join(saved_graph_dir, meta_filename)
print(meta_filepath)

../../../tmp/trt_end_to_end/convert/tf/saver/resnet50/001/tf_model_resnet50_epoch001_2019-09-03T19:31
../../../tmp/trt_end_to_end/convert/tf/saver/resnet50/001/tf_model_resnet50_epoch001_2019-09-03T19:31.meta


Output: Frozen Graph

In [9]:
frozen_graph_dir = get_frozen_graph_dir(_NAME, _EPOCH)

frozen_graph_filename = FROZEN_GRAPH_FILE_FORMAT % (_NAME, _EPOCH, _TIME)
frozen_graph_filepath = os.path.join(frozen_graph_dir, frozen_graph_filename)
print(frozen_graph_filepath)

../../../tmp/trt_end_to_end/convert/tf/frozen/resnet50/001/resnet50_epoch001_2019-09-03T19:31_frozen.pb


In [10]:
# has to be use this setting to make a session for TensorRT optimization
with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.50))) as sess:
    # import the meta graph of the tensorflow model
    saver = tf.train.import_meta_graph(meta_filepath)
    # then, restore the weights to the meta graph
    saver.restore(sess, saver_filepath)
    
    # specify which tensor output you want to obtain 
    your_outputs = [output.split(':')[0] for output in metadata.output_names]
    
    # convert to frozen model
    frozen_graph = tf.graph_util.convert_variables_to_constants(
        sess, # session
        tf.get_default_graph().as_graph_def(),# graph+weight from the session
        output_node_names=your_outputs)
    #write the TensorRT model to be used later for inference
    with gfile.FastGFile(frozen_graph_filepath, 'wb') as f:
        f.write(frozen_graph.SerializeToString())
    print("Frozen model is successfully stored to %s" % frozen_graph_filepath)

W0904 00:10:01.614046 140372402968384 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
W0904 00:10:02.934310 140372402968384 deprecation.py:323] From <ipython-input-10-a7d3bf5bdac7>:15: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W0904 00:10:02.935462 140372402968384 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/graph_util_impl.py:270: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.com

Frozen model is successfully stored to ../../../tmp/trt_end_to_end/convert/tf/frozen/resnet50/001/resnet50_epoch001_2019-09-03T19:31_frozen.pb
